# Regularité mensuelle des trains intercités : modèle de regression linéaire

Ce notebook a pour objectif d'étudier la possibilité de prédire, à partir de données portant sur les trains intercités et une cinquantaine de gares en France Métropolitaine, le nombre de trains ayant minimum 5 minutes de retard à l'arrivée en terminus. Les données ont été récoltées à partir du site SNCF Open Data : https://ressources.data.sncf.com/explore/dataset/regularite-mensuelle-intercites/information/?sort=date

## Importation des modules

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

## Importation et traitement des données

In [2]:
regularite = pd.read_csv(r'regularite-mensuelle-intercites.csv', sep = ';')

séléction des colonnes nécessaires pour la construction du modèle

In [3]:
regularite = regularite[['Date', 'Axe', 'Départ', 'Arrivée', 'Nombre de trains programmés',
       "Nombre de trains en retard à l'arrivée"]]

Suppression des données manquantes

In [4]:
regularite.dropna(inplace = True)

Aperçu des données

In [5]:
regularite

,Date,Axe,Départ,Arrivée,Nombre de trains programmés,Nombre de trains en retard à l'arrivée
0,2014-01,Nord-Normandie,PARIS-ST-LAZARE,CHERBOURG,173.0,14.0
1,2014-01,Nord-Normandie,LE-HAVRE,PARIS-ST-LAZARE,364.0,33.0
2,2014-01,Nord-Normandie,MAUBEUGE,PARIS-NORD,306.0,18.0
3,2014-01,Nord-Normandie,CAEN,PARIS-ST-LAZARE,193.0,20.0
4,2014-01,Centre Sud-Ouest,ORLÉANS,PARIS-AUSTERLITZ,355.0,40.0
...,...,...,...,...,...,...
3800,2020-07,Méditerranée Atlantique,TOULOUSE-MATABIAU,HENDAYE,123.0,23.0
3801,2020-07,Paris Sud,CAHORS/BRIVE,PARIS-AUSTERLITZ,174.0,16.0
3802,2020-07,Paris Sud,CERBERE/TOULOUSE,PARIS-AUSTERLITZ,21.0,1.0
3803,2020-07,Paris Sud,PARIS-AUSTERLITZ,RODEZ/ALBI,10.0,2.0


Création d'une colonne 'saison' à partir de la colonne 'Date'

In [6]:
saisons = {'01' : 'hiver', '02' : 'hiver', '03' : 'hiver',
           '04' : 'printemps', '05' : 'printemps', '06' : 'printemps',
           '07' : 'été', '08' : 'été', '09' : 'été',
           '10' : 'automne', '11' : 'automne', '12' : 'automne'}

regularite['Date'] = regularite['Date'].apply(lambda x : x.split('-')[1])
regularite['Saison'] = regularite['Date'].apply(lambda x : saisons[x])
regularite.drop(columns = 'Date', inplace = True)

In [7]:
regularite[['Saison', 'Axe', 'Départ', 'Arrivée', 'Nombre de trains programmés', "Nombre de trains en retard à l'arrivée"]]

,Saison,Axe,Départ,Arrivée,Nombre de trains programmés,Nombre de trains en retard à l'arrivée
0,hiver,Nord-Normandie,PARIS-ST-LAZARE,CHERBOURG,173.0,14.0
1,hiver,Nord-Normandie,LE-HAVRE,PARIS-ST-LAZARE,364.0,33.0
2,hiver,Nord-Normandie,MAUBEUGE,PARIS-NORD,306.0,18.0
3,hiver,Nord-Normandie,CAEN,PARIS-ST-LAZARE,193.0,20.0
4,hiver,Centre Sud-Ouest,ORLÉANS,PARIS-AUSTERLITZ,355.0,40.0
...,...,...,...,...,...,...
3800,été,Méditerranée Atlantique,TOULOUSE-MATABIAU,HENDAYE,123.0,23.0
3801,été,Paris Sud,CAHORS/BRIVE,PARIS-AUSTERLITZ,174.0,16.0
3802,été,Paris Sud,CERBERE/TOULOUSE,PARIS-AUSTERLITZ,21.0,1.0
3803,été,Paris Sud,PARIS-AUSTERLITZ,RODEZ/ALBI,10.0,2.0


## Encodage des données catégoriques

fonction encodant une colonne puis la transformant en matrice

In [8]:
def transform_categorical_column(df, column):
    df_col = df[[column]]
    df_col = pd.get_dummies(df_col)
    X = np.array(df_col)
    return(X)

Application pour chaque colonne, puis concaténation des matrices pour former l'ensemble des variables prédictives

In [9]:
X_saison = transform_categorical_column(regularite, 'Saison')
X_axe = transform_categorical_column(regularite, 'Axe')
X_depart = transform_categorical_column(regularite, 'Départ')
X_arrivee = transform_categorical_column(regularite, 'Arrivée')
X_prog = np.array(regularite[['Nombre de trains programmés']])

X = np.concatenate((X_saison, X_axe, X_depart, X_arrivee, X_prog), axis = 1)

print(X)

[[  0.   1.   0. ...   0.   0. 173.]
 [  0.   1.   0. ...   0.   0. 364.]
 [  0.   1.   0. ...   0.   0. 306.]
 ...
 [  0.   0.   0. ...   0.   0.  21.]
 [  0.   0.   0. ...   0.   0.  10.]
 [  0.   0.   0. ...   0.   0.  10.]]


Variables à prédire, correspondant à la colonne "Nombre de trains en retard à l'arrivée"

In [10]:
y = np.array(regularite[["Nombre de trains en retard à l'arrivée"]])
print(y)

[[14.]
 [33.]
 [18.]
 ...
 [ 1.]
 [ 2.]
 [ 1.]]


Création des échantillons de test et d'apprentissage

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
print("taille de l'echantillon d'apprentissage : ", len(X_train))
print("taille de l'echantillon de test : ", len(X_test))

taille de l'echantillon d'apprentissage :  3040
taille de l'echantillon de test :  760


Application du modèle de regression linéaire multiple

In [12]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [13]:
y_pred = regressor.predict(X_test)

Comparatif valeurs prédites vs valeurs réelles

In [14]:
np.set_printoptions(precision=2)
pd.DataFrame(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1)).rename(columns = {0 : 'y_pred', 1 : 'y_test'})

,y_pred,y_test
0,11.358902,6.0
1,8.805450,8.0
2,32.502525,19.0
3,15.813911,18.0
4,1.887703,1.0
...,...,...
755,27.811409,33.0
756,29.717987,32.0
757,11.875237,18.0
758,24.294441,38.0


Score du modèle

In [15]:
regressor.score(X, y)

0.7070199949958398

Ainsi, on obtient un score de précision de 0.71 environ.

## Pistes pour améliorer la prédiction

* Coupler ces données avec d'autres données pouvant avoir une incidence sur le retard des trains, telles que  les périodes de grève
* appliquer d'autres modèles tels que les arbres de décision ou Random Forest